# Getting Started

## Introduction
ReSyPE (pronounced *recipe*) is a Python library built for both practitioners and researchers that wish to quickly develop and deploy ML-based recommender systems.

The library provides an end-to-end pipeline that includes:

1. Loading transaction, user feature, and item feature datasets
2. Interchangable methods for user and item clustering
3. Modular framework for machine learning models
4. Iterative and decomposition-based techniques


## Installation

`pip install resype`


## Load the Data

In [44]:
import pandas as pd
import numpy as np
from resype.resype import Resype

%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
# load transaction list
transaction_list = pd.read_csv("sample_data/ratings.csv")[['userId', 'movieId', 'rating']]
transaction_list = transaction_list.sample(20)
transaction_list.columns = ["user_id", 'item_id', 'rating']

## Preprocess

In [46]:
re = Resype(transaction_list)
re.transaction_list.head()

,user_id,item_id,rating
99088,608,2761,4.5
100220,610,48780,4.0
86761,561,2478,4.0
42480,288,2078,3.0
16822,105,112552,4.0


In [47]:
# construct utlity matrix
re.construct_utility_matrix()

item_id,474,1527,2078,2478,2572,2694,2761,2993,3785,4638,4993,5944,5954,5989,8874,48780,85736,95873,112552,136471
user_id,,,,,,,,,,,,,,,,,,,,
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5
135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Train on Unclustered Matrix

### Iterative Approach

In [48]:
# import sklearn Model
from sklearn.neural_network import MLPClassifier, MLPRegressor
mlp1 = MLPRegressor(hidden_layer_sizes=(50, 50))

In [49]:
# fit and predict
re.fit(mlp1, method='iterative')
re.utility_matrix_preds.head()

item_id,474,1527,2078,2478,2572,2694,2761,2993,3785,4638,4993,5944,5954,5989,8874,48780,85736,95873,112552,136471
user_id,,,,,,,,,,,,,,,,,,,,
64,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.007312,0.0,0.0,0.0,0.000000,0.000000
83,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-0.002213,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
91,0.0,-0.030546,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
105,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.041852,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.050566,0.000000
111,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.024515


In [54]:
# recommend
user_list = [0, 1, 2] # indices
top_n = 10
re.get_rec(user_list, top_n)

,user_id,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
0,0,19.0,18.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
1,1,19.0,18.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
2,2,19.0,18.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0


### SVD Approach

In [57]:
# fit and predict
re.fit(mlp1, method='svd')
re.utility_matrix_preds.head()

item_id,474,1527,2078,2478,2572,2694,2761,2993,3785,4638,4993,5944,5954,5989,8874,48780,85736,95873,112552,136471
user_id,,,,,,,,,,,,,,,,,,,,
64,-0.001245,0.007033,0.050139,-0.043433,-0.008777,0.053417,0.01109,0.01181,-0.028744,0.033077,-0.049808,0.056961,0.021974,0.019027,0.000000,-0.268174,0.056766,0.307513,-0.07265,0.009902
83,-0.001245,0.007033,0.050139,-0.043433,-0.008777,0.053417,0.01109,0.01181,-0.028744,0.033077,0.000000,0.056961,0.021974,0.019027,0.037833,-0.268174,0.056766,0.307513,-0.07265,0.009902
91,-0.001245,0.000000,0.050139,-0.043433,-0.008777,0.053417,0.01109,0.01181,-0.028744,0.033077,-0.049808,0.056961,0.021974,0.019027,0.037833,-0.268174,0.056766,0.307513,-0.07265,0.009902
105,-0.001245,0.007033,0.050139,-0.043433,-0.008777,0.053417,0.01109,0.01181,0.000000,0.033077,-0.049808,0.056961,0.021974,0.019027,0.037833,-0.268174,0.056766,0.307513,0.00000,0.009902
111,-0.001245,0.007033,0.050139,-0.043433,-0.008777,0.053417,0.01109,0.01181,-0.028744,0.033077,-0.049808,0.056961,0.021974,0.019027,0.037833,-0.268174,0.056766,0.307513,-0.07265,0.000000


In [58]:
# recommend
user_list = [0, 1, 2] # indices
top_n = 10
re.get_rec(user_list, top_n)

,user_id,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
0,0,17.0,11.0,16.0,5.0,2.0,9.0,12.0,13.0,7.0,6.0
1,1,17.0,11.0,16.0,5.0,2.0,14.0,9.0,12.0,13.0,7.0
2,2,17.0,11.0,16.0,5.0,2.0,14.0,9.0,12.0,13.0,7.0


## Train on Clustered matrix